In [73]:
import numpy as np
import pandas as pd
import sys
np.set_printoptions(threshold = sys.maxsize)

In [74]:
#from google.colab import files
#uploaded = files.upload()

In [75]:
#import io
#train = io.BytesIO(uploaded["train.csv"])
#test = io.BytesIO(uploaded["test.csv"])


In [76]:
#test = pd.read_csv(test)
#test_copy = test.copy()
import os

for dir_path, dir_name, file_name in os.walk(r'C:\Users\Home\Desktop\currentfolders\CrossSellPrediction\data'):
    for file in file_name:
        print(os.path.join(dir_path, file))
    

C:\Users\Home\Desktop\currentfolders\CrossSellPrediction\data\test.csv
C:\Users\Home\Desktop\currentfolders\CrossSellPrediction\data\train.csv


In [77]:
test = pd.read_csv(r"C:\Users\Home\Desktop\currentfolders\CrossSellPrediction\data\test.csv")
train = pd.read_csv(r"C:\Users\Home\Desktop\currentfolders\CrossSellPrediction\data\train.csv")
#train.info()

In [78]:
test_copy = test.copy()
train["is_train"] = "True"
test["is_train"] = "False"

pilot_train = pd.concat([train, test])

In [79]:
#column names and its types

pilot_train = pilot_train.reset_index()
#pilot_train.info()

In [80]:
#pilot_train

train_copy = train.copy()

In [81]:
#converting integer columns to cateogical columns

#col_names = ["Driving_License", "Region_Code", "Previously_Insured", "Policy_Sales_Channel", "Response"]
#pilot_train[col_names] = pilot_train[col_names].astype("category")

In [82]:
#check for common id values in train and test

#set(train["id"]).intersection(set(test["id"]))
#there exists no common train and test id

In [83]:
#Checking for null values in each column 

#(train.isnull().sum()) / len(train)
#there exists no null values in the training dataset

In [84]:
#categorical variables and its unique_values

cat_columns = ["Gender", "Driving_License", "Previously_Insured", "Vehicle_Age", "Vehicle_Damage"]
for cols in cat_columns:
      print("the unique values of", cols, "is", pilot_train[cols].unique())


the unique values of Gender is ['Male' 'Female']
the unique values of Driving_License is [1 0]
the unique values of Previously_Insured is [0 1]
the unique values of Vehicle_Age is ['> 2 Years' '1-2 Year' '< 1 Year']
the unique values of Vehicle_Damage is ['Yes' 'No']


In [85]:
#Vintage column (in days)

#pilot_train["Vintage"].describe()

In [86]:
#Vehicle_age mapping

gender = {'Male' : 0,
          'Female': 1}
          
va = {'> 2 Years' : 2,
      '1-2 Year' : 1,
      '< 1 Year' : 0}

      
vd = {'No' : 0,
      'Yes' : 1}



In [87]:
pilot_train["Gender"] = pilot_train["Gender"].map(gender)
pilot_train["Vehical_Age"] = pilot_train["Vehicle_Age"].map(va)
pilot_train["Vehical_Damage"] = pilot_train["Vehicle_Damage"].map(vd)

In [88]:
#pilot_train.info()

In [89]:
pilot_train["MeanAnnualPremiumPreviouslyInsured"] = pilot_train.groupby(["Previously_Insured"])["Annual_Premium"].transform("mean")
pilot_train["SumAnnualPremiumPreviouslyInsured"] = pilot_train.groupby(["Previously_Insured"])["Annual_Premium"].transform("sum")


In [90]:
pilot_train["Vintage_categories"] = pd.qcut(pilot_train["Vintage"], q = 4, precision = 0, labels = [0, 1, 2, 3])
pilot_train["Vintage_categories"] = pilot_train["Vintage_categories"].astype(int)

In [91]:
#pd.crosstab(pilot_train["Vintage_categories"], pilot_train["Previously_Insured"])

In [92]:
#(pilot_train["Policy_Sales_Channel"].value_counts(normalize = True).sort_values(ascending = False).head(10).index)

In [93]:
popular_PolicySalesChannel = test["Policy_Sales_Channel"].value_counts(normalize = True).sort_values(ascending = False).index
first_10_psc = popular_PolicySalesChannel[0:11]

from collections import defaultdict
first_10_psc_map = defaultdict(int, dict(zip(first_10_psc, np.linspace(10, 1, 10))))
other_policy_channels = popular_PolicySalesChannel[12:] 


#first_10_psc_map
#psc_map = dict(zip())


In [94]:
pilot_train["psc"] = pilot_train["Policy_Sales_Channel"].copy()

In [95]:
#pilot_train["psc"] = np.where(pilot_train["psc"].isin(other_policy_channels), 0, pilot_train["psc"])
pilot_train["psc"] = pilot_train["psc"].map(first_10_psc_map)
#.fillna(pilot_train["psc"])


In [96]:
pilot_train["rc"] = pilot_train["Region_Code"].copy()

In [97]:
popular_regions = pilot_train["Region_Code"].value_counts(normalize = True).sort_values(ascending = False).index
first_10_regions = popular_regions[0: 11]
first_10_regions_map = defaultdict(int, dict(zip(first_10_regions, np.linspace(10, 1, 10))))
other_regions = popular_regions[12:]


In [98]:
#pilot_train["rc"] = np.where(pilot_train["rc"].isin(other_regions), 0, pilot_train["rc"])
pilot_train["rc"] = pilot_train["rc"].map(first_10_regions_map)
#.fillna(pilot_train["rc"])

In [99]:
#pilot_train["rc"].isnull().sum()

In [100]:
#region_codes = (np.array(test["Region_Code"].value_counts(normalize = True).sort_values(ascending = True).head(20).index))
#region_codes_map = dict(zip(region_codes, np.arange(len(region_codes))))
#region_codes_map

In [101]:
#pilot_train["Region_Code_cat"] = pilot_train["Region_Code"].map(region_codes_map)
#pilot_train["Region_Code_cat"].value_counts()
#region_codes_map

In [102]:
#count of previously insured in each region

pilot_train["RegionWiseDepositorsCount"] = pilot_train.groupby(["Region_Code"])["Previously_Insured"].transform("count")
pilot_train["PolicySalesChannelCount"] = pilot_train.groupby(["Policy_Sales_Channel"])["Previously_Insured"].transform("count")
pilot_train["DrivingLicenseCount"] = pilot_train.groupby(["Driving_License"])["Previously_Insured"].transform("count")


In [103]:
#pilot_train = pilot_train.drop(["MeanAnnualPremiumPolicySales", 
#                                "SumAnnualPremiumPolicySales", 
#                                "MinAnnualPremiumPolicySales", 
#                                "MaxAnnualPremiumPolicySales"], axis = 1)

In [104]:
#Mean, sum, max and min Annual Premium of Policy Sales Channel

#pilot_train["MeanAnnualPremiumRegion"] = pilot_train.groupby(["Region_Code"])["Annual_Premium"].transform("mean")
#pilot_train["SumAnnualPremiumRegion"] = pilot_train.groupby(["Region_Code"])["Annual_Premium"].transform("sum")
#pilot_train["MinAnnualPremiumRegion"] = pilot_train.groupby(["Region_Code"])["Annual_Premium"].transform("min")
#pilot_train["MaxAnnualPremiumRegion"] = pilot_train.groupby(["Region_Code"])["Annual_Premium"].transform("max")


In [105]:
pilot_train["MeanAnnualPremiumRegion"] = pilot_train.groupby(["rc"])["Annual_Premium"].transform("mean")
pilot_train["SumAnnualPremiumRegion"] = pilot_train.groupby(["rc"])["Annual_Premium"].transform("sum")
pilot_train["MinAnnualPremiumRegion"] = pilot_train.groupby(["rc"])["Annual_Premium"].transform("min")
pilot_train["MaxAnnualPremiumRegion"] = pilot_train.groupby(["rc"])["Annual_Premium"].transform("max")


In [106]:
#Mean, sum, max and min of Annual Premium Regionwise

#pilot_train["MeanAnnualPremiumPolicySales"] = pilot_train.groupby(["Policy_Sales_Channel"])["Annual_Premium"].transform("mean")
#pilot_train["SumAnnualPremiumPolicySales"] = pilot_train.groupby(["Policy_Sales_Channel"])["Annual_Premium"].transform("sum")
#pilot_train["MinAnnualPremiumPolicySales"] = pilot_train.groupby(["Policy_Sales_Channel"])["Annual_Premium"].transform("min")
#pilot_train["MaxAnnualPremiumPolicySales"] = pilot_train.groupby(["Policy_Sales_Channel"])["Annual_Premium"].transform("max")


In [107]:
pilot_train["MeanAnnualPremiumPolicySales"] = pilot_train.groupby(["psc"])["Annual_Premium"].transform("mean")
pilot_train["SumAnnualPremiumPolicySales"] = pilot_train.groupby(["psc"])["Annual_Premium"].transform("sum")
pilot_train["MinAnnualPremiumPolicySales"] = pilot_train.groupby(["psc"])["Annual_Premium"].transform("min")
pilot_train["MaxAnnualPremiumPolicySales"] = pilot_train.groupby(["psc"])["Annual_Premium"].transform("max")


In [108]:
#Feature names

#pilot_train.columns

In [109]:
pilot_train.drop(["index", "id", "Vehicle_Age", "Vehicle_Damage"], axis = 1, inplace = True)


In [110]:

features = ['Gender', 'Age', 'Driving_License',
       'Previously_Insured',  'Annual_Premium',
         'Response', 'is_train',
       'Vehical_Age', 'Vehical_Damage', 'MeanAnnualPremiumPreviouslyInsured',
       'SumAnnualPremiumPreviouslyInsured', 'Vintage_categories', 'psc', 'rc',
       'RegionWiseDepositorsCount', 'MeanAnnualPremiumRegion',
       'SumAnnualPremiumRegion', 'MinAnnualPremiumRegion',
       'MaxAnnualPremiumRegion', 'MeanAnnualPremiumPolicySales',
       'SumAnnualPremiumPolicySales', 'MinAnnualPremiumPolicySales',
       'MaxAnnualPremiumPolicySales']

In [111]:
train_data = pilot_train.loc[pilot_train["is_train"].isin(["True"])]
test_data = pilot_train.loc[pilot_train["is_train"].isin(["False"])]
train_data = train_data[features]
test_data = test_data[features]

In [112]:
#train_data.info()

In [113]:
#pilot_train.columns
import lightgbm as lgb

In [114]:
#pilot_train.info()

#pilot_train.info()

x = train_data.drop(["Response", "is_train"], axis = 1).reset_index()
y = train_data.loc[:, ["Response"]]
test = test_data.drop(["Response", "is_train"], axis = 1).reset_index()
#test.shape

In [115]:
#x.shape, y.shape, test.shape

In [116]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
y_pred_lgbm = np.zeros((len(test), 2))
#train_copy = pilot_train[features]
#train_Y = pilot_train.loc[:, ["Response"]]
kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 3)
i = 1
scores = []
for train_index, test_index in kf.split(x, y):
  x_train, x_test = x.iloc[train_index], x.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  clf = lgb.LGBMClassifier(boosting_type='gbdt', learning_rate=0.1, n_estimators=800, importance_type='gain', objective='binary', num_boost_round=100,
                            num_leaves=400, max_depth=5, max_bin = 60, 
                             bagging_faction=0.9, feature_fraction=0.9, subsample_freq=2, scale_pos_weight=2.5,
                            random_state= 1994, n_jobs=-1, reg_lambda = 1, silent=False, unbalanced = True)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy = accuracy_score(y_test, y_pred)
  print("The accuracy scored for fold", i, "is", accuracy)
  i += 1
  scores.append(accuracy)
  #y_pred_lgbm += clf.predict_prob(test)






C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


The accuracy scored for fold 1 is 0.8137545590511925
The accuracy scored for fold 2 is 0.8196846054944766
The accuracy scored for fold 3 is 0.81711316942615
The accuracy scored for fold 4 is 0.8160373645404214
The accuracy scored for fold 5 is 0.8139644721996274
The accuracy scored for fold 6 is 0.8193172574847157
The accuracy scored for fold 7 is 0.8143318202093883
The accuracy scored for fold 8 is 0.815040277085356
The accuracy scored for fold 9 is 0.8150140379418016
The accuracy scored for fold 10 is 0.8138808711624246


In [502]:
final_test = io.BytesIO(uploaded["test.csv"])
final_test = pd.read_csv(final_test)
final_test.shape


(127037, 11)

In [117]:
submission_data = pd.DataFrame({"id": test_copy["id"], "Response" : clf.predict_proba(test)[:, 1]} )
#submission_data.reset_index(drop = True, inplace = True)
os.chdir(r"C:\Users\Home\Desktop\currentfolders\CrossSellPrediction")
#submission_data["Response"] = clf.predict(test)
#pred_values = list(clf.predict(test))
submission_data.to_csv("CrossSellPrediction.csv")
#files.download("Submission2.csv")
#len(pred_values)
#submission_data["Response"] = pred_values
#submission_data["id"] = len(final_test.id.values)
#submission_data.shape, len(pred_values)
#len(final_test["Vintage"])

In [119]:
#print(test_copy.shape,test.shape)
#len(list(clf.predict(test))), len(test)

In [ ]:
#!pip install catboost
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension